<a href="https://colab.research.google.com/github/qahathaway/Automated_Segmentation_PTM-IMAT-SAT/blob/main/Automated_Segmentation_PM_IMAT_SAT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
### Load Essential Packages for DICOM ---> PNG ###
import cv2
import os
import pydicom
import numpy as np
from skimage import exposure

from pydicom import dcmread
from pydicom.pixel_data_handlers.util import apply_voi_lut

In [ ]:
### Automatically Create PNG from DICOM for All Patients in Directory ###
import glob
from glob import iglob

for filename in glob.iglob('/media/bitlockermount/MESArthritis/**/*.dcm',recursive = True):
    ds = pydicom.read_file(filename) #read dicom image
    img = ds.pixel_array # get image array
    img2 = (np.maximum(img, 925) / img.max()) * 255.0
    img3 = cv2.resize(img2, (12, 12), interpolation = cv2.INTER_NEAREST) 
    cv2.imwrite('/home/john/Documents/MESArthritis/Recursive' + filename.replace('.dcm','.png'),img3) # write png image

In [ ]:
### Automatically Create PNG from DICOM for Individual Patients ###
inputdir = '/media/bitlockermount/MESArthritis/MESA3010007/'
outdir = '/home/john/Documents/MESArthritis/MESA3010007/'

test_list = [ f for f in  os.listdir(inputdir)]

for f in test_list[:10]:   # remove "[:10]" to convert all images 
    ds = pydicom.read_file(inputdir + f) # read dicom image
    img = ds.pixel_array # get image array
    img2 = (np.maximum(img, 925) / img.max()) * 255.0
    img3 = cv2.resize(img2, (1200, 1200), interpolation = cv2.INTER_NEAREST) 
    cv2.imwrite(outdir + f.replace('.dcm','.png'),img3) # write png image

In [ ]:
### Load Essential Packages for Automatic Selection of PNG Directly Above Aortic Arch ###
import numpy as np
import pandas as pd
import cv2
from imutils import paths
import argparse
import pickle
import vptree
import matplotlib.pyplot as plt
import time

In [ ]:
### Create Image Hashing ###
def dhash(image, hashSize=8):
    resized = cv2.resize(image, (hashSize + 1, hashSize))
    diff = resized[:, 1:] > resized[:, :-1]
    return sum([2 ** i for (i, v) in enumerate(diff.flatten()) if v])

### Converting Data Types ###
def convert_hash(h):
    return int(np.array(h, dtype="float64"))

def hamming(a, b):
    return bin(int(a) ^ int(b)).count("1")

In [ ]:
imagePaths = list(paths.list_images("/home/john/Documents/MESArthritis/MESA3010090/"))
hashes= {}

### Loading and Hashing the Images ###
for (i, img_path) in enumerate(imagePaths):
    image = cv2.imread(img_path)
    h = dhash(image)
    h = convert_hash(h)
    l = hashes.get(h, [])
    l.append(img_path)
    hashes[h] = l

In [ ]:
points = list(hashes.keys())
tree = vptree.VPTree(points, hamming) #Creating VP-Tree

In [ ]:
### Serializing VP-Tree ###
f = open("tree.pickle", "wb")
f.write(pickle.dumps(tree))
f.close()

### Serialize the Hashes to Dictionary ###
f = open("hashes.pickle", "wb")
f.write(pickle.dumps(hashes))
f.close()

In [ ]:
tree = pickle.loads(open("tree.pickle", "rb").read())
hashes = pickle.loads(open("hashes.pickle", "rb").read())

### Compute the Hash for the Query Image, then Convert It ###
queryHash = dhash(image)
queryHash = convert_hash(queryHash)

In [ ]:
image = cv2.imread("/home/john/Documents/MESArthritis/IM-0205-0102.png")

In [ ]:
queryHash = dhash(image)
queryHash = convert_hash(queryHash)

In [ ]:
start = time.time()
results = tree.get_all_in_range(queryHash, 13)
results = sorted(results)
end = time.time()
print(">>> search took {} seconds".format(end - start))

In [ ]:
### Loop Over the Results and Select the Most Similar PNG ###
for (d, h) in results[:1]:
    resultPaths = hashes.get(h, [])
    print(">>> {} total image(s) with d: {}, h: {}".format(len(resultPaths), d, h))
    for resultPath in resultPaths:
        result = cv2.imread(resultPath)
        cv2.imwrite('/home/john/Documents/MESArthritis/MESA3010058.png',result)
        plt.imshow(result)
        plt.show()

In [ ]:
### Load Essential Packages for Creating ROI for PM, IMAT, and SAT ###
import os
import sys
import random
import math
import re
import time
import numpy as np
import cv2
import matplotlib
import matplotlib.pyplot as plt

# Root directory of the project
ROOT_DIR = os.path.abspath("../../")

# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
from mrcnn.config import Config
from mrcnn import utils
import mrcnn.model as modellib
from mrcnn import visualize
from mrcnn.model import log

%matplotlib inline 

# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")

# Local path to trained weights file
COCO_MODEL_PATH = os.path.join(ROOT_DIR, "mask_rcnn_coco.h5")
# Download COCO trained weights from Releases if needed
if not os.path.exists(COCO_MODEL_PATH):
    utils.download_trained_weights(COCO_MODEL_PATH)

In [ ]:
class ShapesConfig(Config):
    """Configuration for training on the toy shapes dataset.
    Derives from the base Config class and overrides values specific
    to the toy shapes dataset.
    """
    # Give the configuration a recognizable name
    NAME = "shapes"

    # Train on 1 GPU and 8 images per GPU. We can put multiple images on each
    # GPU because the images are small. Batch size is 8 (GPUs * images/GPU).
    GPU_COUNT = 1
    IMAGES_PER_GPU = 8

    # Number of classes (including background)
    NUM_CLASSES = 1 + 3  # background + 3 shapes

    # Use small images for faster training. Set the limits of the small side
    # the large side, and that determines the image shape.
    IMAGE_MIN_DIM = 128
    IMAGE_MAX_DIM = 128

    # Use smaller anchors because our image and objects are small
    RPN_ANCHOR_SCALES = (8, 16, 32, 64, 128)  # anchor side in pixels

    # Reduce training ROIs per image because the images are small and have
    # few objects. Aim to allow ROI sampling to pick 33% positive ROIs.
    TRAIN_ROIS_PER_IMAGE = 32

    # Use a small epoch since the data is simple
    STEPS_PER_EPOCH = 100

    # use small validation steps since the epoch is small
    VALIDATION_STEPS = 5
    
config = ShapesConfig()
config.display()

In [ ]:
def get_ax(rows=1, cols=1, size=8):
    """Return a Matplotlib Axes array to be used in
    all visualizations in the notebook. Provide a
    central point to control graph sizes.
    
    Change the default size attribute to control the size
    of rendered images
    """
    _, ax = plt.subplots(rows, cols, figsize=(size*cols, size*rows))
    return ax

In [ ]:
class ShapesDataset(utils.Dataset):
    """Generates the shapes synthetic dataset. The dataset consists of simple
    shapes (triangles, squares, circles) placed randomly on a blank surface.
    The images are generated on the fly. No file access required.
    """

    def load_shapes(self, count, height, width):
        """Generate the requested number of synthetic images.
        count: number of images to generate.
        height, width: the size of the generated images.
        """
        # Add classes
        self.add_class("shapes", 1, "square")
        self.add_class("shapes", 2, "circle")
        self.add_class("shapes", 3, "triangle")

        # Add images
        # Generate random specifications of images (i.e. color and
        # list of shapes sizes and locations). This is more compact than
        # actual images. Images are generated on the fly in load_image().
        for i in range(count):
            bg_color, shapes = self.random_image(height, width)
            self.add_image("shapes", image_id=i, path=None,
                           width=width, height=height,
                           bg_color=bg_color, shapes=shapes)

    def load_image(self, image_id):
        """Generate an image from the specs of the given image ID.
        Typically this function loads the image from a file, but
        in this case it generates the image on the fly from the
        specs in image_info.
        """
        info = self.image_info[image_id]
        bg_color = np.array(info['bg_color']).reshape([1, 1, 3])
        image = np.ones([info['height'], info['width'], 3], dtype=np.uint8)
        image = image * bg_color.astype(np.uint8)
        for shape, color, dims in info['shapes']:
            image = self.draw_shape(image, shape, dims, color)
        return image

    def image_reference(self, image_id):
        """Return the shapes data of the image."""
        info = self.image_info[image_id]
        if info["source"] == "shapes":
            return info["shapes"]
        else:
            super(self.__class__).image_reference(self, image_id)

    def load_mask(self, image_id):
        """Generate instance masks for shapes of the given image ID.
        """
        info = self.image_info[image_id]
        shapes = info['shapes']
        count = len(shapes)
        mask = np.zeros([info['height'], info['width'], count], dtype=np.uint8)
        for i, (shape, _, dims) in enumerate(info['shapes']):
            mask[:, :, i:i+1] = self.draw_shape(mask[:, :, i:i+1].copy(),
                                                shape, dims, 1)
        # Handle occlusions
        occlusion = np.logical_not(mask[:, :, -1]).astype(np.uint8)
        for i in range(count-2, -1, -1):
            mask[:, :, i] = mask[:, :, i] * occlusion
            occlusion = np.logical_and(occlusion, np.logical_not(mask[:, :, i]))
        # Map class names to class IDs.
        class_ids = np.array([self.class_names.index(s[0]) for s in shapes])
        return mask.astype(np.bool), class_ids.astype(np.int32)

    def draw_shape(self, image, shape, dims, color):
        """Draws a shape from the given specs."""
        # Get the center x, y and the size s
        x, y, s = dims
        if shape == 'square':
            cv2.rectangle(image, (x-s, y-s), (x+s, y+s), color, -1)
        elif shape == "circle":
            cv2.circle(image, (x, y), s, color, -1)
        elif shape == "triangle":
            points = np.array([[(x, y-s),
                                (x-s/math.sin(math.radians(60)), y+s),
                                (x+s/math.sin(math.radians(60)), y+s),
                                ]], dtype=np.int32)
            cv2.fillPoly(image, points, color)
        return image

    def random_shape(self, height, width):
        """Generates specifications of a random shape that lies within
        the given height and width boundaries.
        Returns a tuple of three valus:
        * The shape name (square, circle, ...)
        * Shape color: a tuple of 3 values, RGB.
        * Shape dimensions: A tuple of values that define the shape size
                            and location. Differs per shape type.
        """
        # Shape
        shape = random.choice(["square", "circle", "triangle"])
        # Color
        color = tuple([random.randint(0, 255) for _ in range(3)])
        # Center x, y
        buffer = 20
        y = random.randint(buffer, height - buffer - 1)
        x = random.randint(buffer, width - buffer - 1)
        # Size
        s = random.randint(buffer, height//4)
        return shape, color, (x, y, s)

    def random_image(self, height, width):
        """Creates random specifications of an image with multiple shapes.
        Returns the background color of the image and a list of shape
        specifications that can be used to draw the image.
        """
        # Pick random background color
        bg_color = np.array([random.randint(0, 255) for _ in range(3)])
        # Generate a few random shapes and record their
        # bounding boxes
        shapes = []
        boxes = []
        N = random.randint(1, 4)
        for _ in range(N):
            shape, color, dims = self.random_shape(height, width)
            shapes.append((shape, color, dims))
            x, y, s = dims
            boxes.append([y-s, x-s, y+s, x+s])
        # Apply non-max suppression wit 0.3 threshold to avoid
        # shapes covering each other
        keep_ixs = utils.non_max_suppression(np.array(boxes), np.arange(N), 0.3)
        shapes = [s for i, s in enumerate(shapes) if i in keep_ixs]
        return bg_color, shapes